# 使用Yi的API构建RAG(LangChain)

在本教程中，我们将学习如何使用LangChain和Yi的API构建一个RAG（Retrieval-Augmented Generation）系统。

## 安装必要的库

首先，我们需要安装LangChain库。

In [ ]:
!pip install langchain

## 配置环境变量

接下来，我们需要配置LangSmith和Yi的API密钥。请确保您已经在[LangSmith](https://smith.langchain.com/)和零一万物[开放平台](https://platform.lingyiwanwu.com/apikeys)注册并获取了API密钥。

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "your_langsmith_api_key"
os.environ["YI_API_KEY"] = "your_yi_api_key"

## 安装LangChain-OpenAI库

我们还需要安装`langchain-openai`库。

In [ ]:
!pip install -qU langchain-openai

## 配置LLM

现在我们来配置LLM，使用Yi的大型语言模型。

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://api.lingyiwanwu.com/v1",
    api_key=os.environ["YI_API_KEY"],
    model="yi-large",
)

## 加载数据

接下来我们将加载一些示例数据。这里我们使用了LangChain的WebBaseLoader来加载网页数据。

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

## 构建向量数据库

我们将使用HuggingFace的嵌入模型来构建向量数据库，并使用Chroma来存储向量。

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 加载向量模型
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

# 切分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 构建向量数据库
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = vectorstore.as_retriever()

## 构建RAG链

最后，我们将构建RAG链，并使用LangChain的hub来获取提示。

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke("What is Task Decomposition?")
print(response)